In [ ]:
%load_ext lab_black
import os
import pickle
import networkx as nx
import community as community_louvain
from IPython.core.display import display, HTML
from time import time

from import_and_scrape import get_exported_youtube_playlist_ids, scrape, id_to_url

# import matplotlib.cm as cm
# import matplotlib.pyplot as plt

In [ ]:
# temporary autocompletion bug fix
%config Completer.use_jedi = False

In [ ]:
graph_path = "../data/graph.pickle"


def save_graph(G):
    assert 0 == len(list(nx.selfloop_edges(G)))
    # selfloop edges shouldn't happen
    # G.remove_edges_from(nx.selfloop_edges(G))

    with open(graph_path, "wb") as handle:
        pickle.dump(G, handle, protocol=pickle.HIGHEST_PROTOCOL)


def load_graph():
    # load or create a Graph
    if os.path.isfile(graph_path):
        print("graph loaded")
        with open(graph_path, "rb") as handle:
            return pickle.load(handle)
    else:
        print("graph created")
        return nx.DiGraph()

In [ ]:
G = load_graph()

In [ ]:
# ids_to_add = get_freetube_favorites_ids()
ids_to_add, times_liked = get_exported_youtube_playlist_ids(
    "../data/youtube-export/playlists/Liked videos.csv"
)

print(f"adding {len(ids_to_add)} nodes")

for id_, time_liked in zip(ids_to_add, times_liked):
    print(">", end="")

    #     if id_ in G.nodes and "time_scraped" in G.nodes[id_]:
    #         # this video was already scraped
    #         continue

    scrape(id_, G)
    if id_ in G:
        G.nodes[id_]["time_liked"] = time_liked

In [ ]:
save_graph(G)

In [ ]:
def get_liked_from_time_range(G, start, end=None):
    if end is None:
        end = time()
    return [
        id_
        for id_, node in G.nodes.data()
        if "time_liked" in node and start < node["time_liked"] < end
    ]


def sort_nodes_by_in_degree(G, nodes):
    recs = sorted(G.subgraph(nodes).in_degree(), key=lambda pair: pair[1], reverse=True)
    ids, scores = zip(*recs)
    return ids, scores

In [ ]:
# def recommend(SubG, pickiness=0):
#     # this line recommends very normie videos
#     # it's equivalent to pickiness==0
#     # recs = sorted(SubG.in_degree(), key=lambda pair: pair[1], reverse=True)

#     # to be honest, I don't fully understand this part
#     # but it works better than the one on top:
#     # first limit recs only to the best ones
#     # this way, we'll omit most general normie recommendations later
#     best_recs = [node for node, in_degree in SubG.in_degree() if in_degree >= pickiness]
#     #     best_recs = [node for node, degree in SubG.degree() if degree >= pickiness]

#     ids, scores = sort_nodes_by_in_degree(SubG, best_recs)
#     return ids, scores


def display_video_links(G, ids, prefixes=None, show_first=15):
    if prefixes is None:
        prefixes = [""] * len(ids)

    for id_, prefix in list(zip(ids, prefixes))[:show_first]:
        if "title" not in G.nodes[id_] or G.nodes[id_]["title"] is None:
            scrape(id_, G)

        url = id_to_url.format(id_)
        title = G.nodes[id_]["title"]
        display(HTML(f"""<a href="{url}">{prefix} {title}</a>"""))


#     save_graph(G)

In [ ]:
seconds_in_year = 60 * 60 * 24 * 365
start_time = time() - seconds_in_year * 4
sources = get_liked_from_time_range(G, start_time)
print(len(sources))
# for id_ in sources:
#     print(G.nodes[id_]["title"])

RecentDirected = G.edge_subgraph(G.out_edges(sources))
Recent = RecentDirected.to_undirected()

In [ ]:
# # prune

# old_nodes_num = -1

# while old_nodes_num != len(Recent.nodes):
#     old_nodes_num = len(Recent.nodes)
#     print(old_nodes_num)
#     pruned_nodes = [node for node, degree in Recent.degree() if degree >= 2]
#     Recent = Recent.subgraph(pruned_nodes)

In [ ]:
dendrogram = community_louvain.generate_dendrogram(Recent)

In [ ]:
partition = community_louvain.partition_at_level(dendrogram, 2)

cluster_numbers = set(partition.values())
len(cluster_numbers)

In [ ]:
cluster = [node for node, label in partition.items() if label == 11]

ids, scores = sort_nodes_by_in_degree(G, cluster)
display_video_links(G, ids, scores)

In [ ]:
Induced = community_louvain.induced_graph(partition, Recent)
Induced.remove_edges_from(nx.selfloop_edges(Induced))
# pos = nx.spring_layout(Induced, k=1)
# nx.draw(Induced, pos=pos, node_size=3)

In [ ]:
salient_edges = []
for u, v, attributes in Induced.edges.data():
    if attributes["weight"] >= 5:
        salient_edges.append((u, v))

nx.draw(nx.Graph(salient_edges), node_size=3)

In [ ]:
best_recs = [node for node, in_degree in G.in_degree() if in_degree >= 12]
nx.draw(G.subgraph(best_recs))

In [ ]:
for n in list(components[0]):
    print(nx.algorithms.distance_measures.eccentricity(S.to_undirected(), v=n))